In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
dataframe = pd.read_csv('train.csv')
dataframe.head()
le = preprocessing.LabelEncoder()
dataframe = dataframe.apply(le.fit_transform)

y = dataframe['Survived']
X = dataframe.drop(['Survived', 'Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state=9)

In [34]:
bagging_clf1 = BaggingClassifier(LogisticRegression(), n_estimators=100, max_samples=100, 
                                bootstrap=True, random_state=9)

bagging_clf1.fit(X_train, y_train)
y_pred_bagging = bagging_clf1.predict(X_test)
score_bc_lr = accuracy_score(y_test, y_pred_bagging)

bagging_clf2 = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, max_samples=100,
                                bootstrap=True, random_state=9)

bagging_clf2.fit(X_train, y_train)
y_pred_bagging = bagging_clf2.predict(X_test)
score_bc_dt = accuracy_score(y_test, y_pred_bagging)


#use soft classifier for combining bagging_clf1 and bagging_clf2 to get predicted y on X_test.
#As discussed above, use this output as target in the meta classifier.

voting_clf_soft = VotingClassifier(estimators = [('bagging_lr', bagging_clf1), 
                                                 ('bagging_dt', bagging_clf2)],voting='soft')
voting_clf_soft.fit(X_train, y_train)
voting_prediction = voting_clf_soft.predict(X_test)
score_voting = accuracy_score(voting_prediction, y_test)

In [26]:
print score_bc_dt
print score_bc_lr
print score_voting

0.782122905028
0.748603351955
0.787709497207


In [37]:
predicted1 = pd.DataFrame(bagging_clf1.predict_proba(X_test))
predicted2 = pd.DataFrame(bagging_clf2.predict_proba(X_test))
predicted3 = pd.DataFrame(voting_clf_soft.predict_proba(X_test))
predicted4 = pd.DataFrame(voting_clf_soft.predict(X_test))

In [38]:
print predicted1.head()
print predicted1.shape

print predicted2.head()
print predicted2.shape

print predicted3.head()
print predicted3.shape

print predicted4.head()
print predicted4.shape

          0         1
0  0.434946  0.565054
1  0.875664  0.124336
2  0.833414  0.166586
3  0.511722  0.488278
4  0.461716  0.538284
(179, 2)
       0      1
0  0.500  0.500
1  0.960  0.040
2  0.915  0.085
3  0.850  0.150
4  0.400  0.600
(179, 2)
          0         1
0  0.467473  0.532527
1  0.917832  0.082168
2  0.874207  0.125793
3  0.680861  0.319139
4  0.430858  0.569142
(179, 2)
   0
0  1
1  0
2  0
3  0
4  1
(179, 1)


In [39]:
p1 = predicted1[0]
p2 = predicted2[0]
print p1.shape
print p2.shape

(179L,)
(179L,)


In [40]:
meta_x = pd.concat([p1, p2], axis=1)
meta_y = predicted4[0]

In [44]:
meta_clf = DecisionTreeClassifier(criterion='gini')
meta_fit = meta_clf.fit(meta_x,meta_y)
meta_predict = meta_clf.predict(predicted3)

In [45]:
accuracy_score(meta_predict, y_test)

0.74860335195530725